# AlgoBulls Assignment

### Import required packages

In [1]:
import numpy as np
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")

### 1. Define a Class ScriptData which can fetch US Stock data using Alpha Vantage.

In [25]:
class ScriptData:
    
    def __init__(self):
        self.info1 = {}   # dict to keep equity as keys and data as value
        self.data1 = {}   # dict to keep equity as keys and dataframe as value
        
    def __getitem__(self, script):
        try:
            return self.data1[script]
        except Exception as e:
            print("Convert the data into dataframe as second step")
    
    def __setitem__(self, script, df):
        self.data1[script] = df
        
    def __contains__(self, script):
        return script in self.data1.keys()
    
    def fetch_intraday_data(self,script):
        try:
            url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol='+script+'&interval=5min&apikey=1912UMDVJ3T55DBL'
            r = requests.get(url)
            data = r.json()     # stores data in json format
            keys = list(data.keys())
            self.info1[script] = data[keys[1]]     # adds data in dict info1, leaves out metadata
        except Exception as e:
            print("No such equity found")
        
    def convert_intraday_data(self,script):
        try:
            df = pd.DataFrame(self.info1[script].values())     # converts values of dict within info1 into dataframe
            df.columns = ['open', 'high', 'low', 'close', 'volume']
            df['timestamp'] = self.info1[script].keys()     # adds timestamp column from keys of the dict within info1
            df = df[['timestamp','open','high','low','close','volume']]     # reaarange columns of dataframe
            df = df[::-1]
            
            # change datatypes of columns
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            df[['open', 'high', 'low', 'close']] = df[['open', 'high', 'low', 'close']].astype(float)
            df['volume'] = df['volume'].astype(int)
            
            self.__setitem__(script, df)     # setitem method of ScripData class
            
        except Exception as e:
            print("Fetch the mentioned equity as the first step")

In [26]:
script_data = ScriptData()     # create instance of class ScriptData

In [27]:
script_data.fetch_intraday_data('AAPL')

In [28]:
script_data.convert_intraday_data('AAPL')

In [29]:
script_data['AAPL']

,timestamp,open,high,low,close,volume
99,2023-06-16 11:15:00,186.050,186.060,185.8699,185.87,390508
98,2023-06-16 11:20:00,185.865,185.920,185.5400,185.68,466449
97,2023-06-16 11:25:00,185.690,185.780,185.3400,185.48,495189
96,2023-06-16 11:30:00,185.480,185.605,185.3800,185.42,481608
95,2023-06-16 11:35:00,185.415,185.565,185.3000,185.52,415376
...,...,...,...,...,...,...
4,2023-06-16 19:35:00,185.000,185.000,185.0000,185.00,822
3,2023-06-16 19:40:00,184.980,185.000,184.9800,184.99,1121
2,2023-06-16 19:45:00,185.000,185.000,184.9900,184.99,2045
1,2023-06-16 19:50:00,184.990,184.990,184.9900,184.99,333


In [7]:
script_data.fetch_intraday_data('GOOGL')

In [8]:
script_data.convert_intraday_data('GOOGL')

In [9]:
script_data['GOOGL']

,timestamp,open,high,low,close,volume
99,2023-06-16 10:45:00,124.270,124.280,123.98,124.0900,207380
98,2023-06-16 10:50:00,124.085,124.140,123.92,124.0800,459559
97,2023-06-16 10:55:00,124.090,124.480,124.08,124.3600,164717
96,2023-06-16 11:00:00,124.360,124.585,124.23,124.4864,231360
95,2023-06-16 11:05:00,124.490,124.750,124.42,124.6750,190252
...,...,...,...,...,...,...
4,2023-06-16 19:30:00,123.700,123.700,123.70,123.7000,1096
3,2023-06-16 19:35:00,123.740,123.740,123.74,123.7400,177
2,2023-06-16 19:40:00,123.700,123.700,123.70,123.7000,504
1,2023-06-16 19:50:00,123.750,123.760,123.70,123.7000,1295


In [10]:
'AAPL' in script_data

True

In [11]:
'GOOGL' in script_data

True

In [12]:
'NVDA' in script_data

False

### 2. Define a function called indicator1.

In [13]:
def indicator1(df, timeperiod):
    try:
        list1 = []   
        for i in range(len(df)+1):     # initial rows with no moving average values
            if i < timeperiod - 1:
                list1.append(None)
            elif i >= timeperiod:     # moving average for sepcified timeperiod
                sum = 0
                for j in range(i-timeperiod, i):
                    sum += df['close'][j]
                mean = sum/timeperiod
                list1.append(mean)
        df2 = pd.DataFrame()
        df2['timestamp'] = df['timestamp']     # adds timestamp column to dataframe
        df2['indicator'] = list1     # adds movings average column to dataframe
        return df2
    except Exception:
            pass

In [14]:
indicator1(script_data['AAPL'], 5)

,timestamp,indicator
99,2023-06-16 11:15:00,NaN
98,2023-06-16 11:20:00,NaN
97,2023-06-16 11:25:00,NaN
96,2023-06-16 11:30:00,NaN
95,2023-06-16 11:35:00,184.982
...,...,...
4,2023-06-16 19:35:00,185.620
3,2023-06-16 19:40:00,185.550
2,2023-06-16 19:45:00,185.514
1,2023-06-16 19:50:00,185.558


In [15]:
indicator1(script_data['GOOGL'], 5)

,timestamp,indicator
99,2023-06-16 10:45:00,NaN
98,2023-06-16 10:50:00,NaN
97,2023-06-16 10:55:00,NaN
96,2023-06-16 11:00:00,NaN
95,2023-06-16 11:05:00,123.69800
...,...,...
4,2023-06-16 19:30:00,124.53600
3,2023-06-16 19:35:00,124.57828
2,2023-06-16 19:40:00,124.57428
1,2023-06-16 19:50:00,124.47828


### 3. Define a Class Strategy. Print the ‘signals’ DataFrame with rows where the signal is either ‘BUY’ or ‘SELL’.

In [16]:
class Strategy:
    
    def __init__(self, script):
        self.script = script
        
    def get_script_data(self):
        try:
            script_data = ScriptData()
            script_data.fetch_intraday_data(self.script)
            script_data.convert_intraday_data(self.script)
            self.df_close = script_data[self.script]     # datagrame with fetched unchanged close column
            self.df_indi = indicator1(self.df_close, 5)     # dataframe with indicator column showing moving average
            self.na = self.df_indi['indicator'].isna().sum()     # total na values in indicator column
        except Exception:
            pass
            
        
    def get_signals(self):
        try:
            signals = pd.DataFrame()     # creates signals dataframe
            signals['timestamp'] = self.df_close['timestamp'][self.na:]     # adds timestamp column with length = length of df_close - total na values in df_indi
            signals['signal'] = 'NO_SIGNAL'     # adds signal column with all values as 'NO_SIGNAL'
        
            if self.df_indi['indicator'][self.na] >= self.df_close['close'][self.na]:     # flag to change the conditions
                flag = True
            else:
                flag = False
            
            for x in range(self.na, len(self.df_indi)):
                if flag == False:
                    if self.df_indi['indicator'][x] > self.df_close['close'][x]:     # if indicator cuts close upwards
                        signals['signal'][x] = 'BUY'     # cell change from 'NO_SIGNAL' to 'BUY'
                        flag = True
                elif flag == True:
                    if self.df_indi['indicator'][x] < self.df_close['close'][x]:     # if indicator cuts close downwards
                        signals['signal'][x] = 'SELL'     # cell change from 'NO_SIGNAL' to 'SELL'
                        flag = False
        
            signals = signals[signals['signal'] != 'NO_SIGNAL']     # filtered dataframe
            return signals
        
        except Exception as e:
            print("No such equity found")

In [17]:
strategy = Strategy('NVDA')     # create instance of class Strategy

In [18]:
strategy.get_script_data()

In [19]:
strategy.get_signals()

,timestamp,signal
49,2023-06-16 15:50:00,SELL
